In [17]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
import torch
from trainer import *
from torch.utils.data import DataLoader as TorchLoader
from transformers import BertModel, BertTokenizer
from model import MiniMoELoadWeights, MiniMoE

In [2]:
tokenizer = BertTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
base_model = BertModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
base_model.config.attention_probs_dropout_prob = 0.05
base_model.config.hidden_dropout_prob = 0.05
#base_model.max_position_embeddings = 2048 # this would be a good addition
loader = MiniMoELoadWeights(base_model=base_model, tokenizer=tokenizer, domains=['[COPD]', '[CVD]', '[HSP90]', '[TGFB]'])
model, tokenizer = loader.get_seeded_model()
mini = MiniMoE(model)

Some weights differ
All weights match
Model loaded in  0.01 minutes
44.0 million total parameters
36.9 million effective parameters
Approximately 0.16 GB of memory in fp32



In [21]:
#copd_dataset.push_to_hub('lhallee/abstract_domain_copd')
#cvd_dataset.push_to_hub('lhallee/abstract_domain_cvd')

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/91 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [5]:
class config:
    data_paths = ['lhallee/abstract_domain_cvd', 'lhallee/abstract_domain_copd']
    epochs = 100
    domains = ['[CVD]', '[COPD]']
    batch_size = 2
    lr = 1e-4
    validate_interval = 10
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
train_dataset, valid_dataset, test_dataset = get_datasets(config.data_paths, tokenizer, config.domains)
train_loader = TorchLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
valid_loader = TorchLoader(valid_dataset, batch_size=config.batch_size, shuffle=True)
test_loader = TorchLoader(test_dataset, batch_size=config.batch_size, shuffle=True)
optimizer = torch.optim.AdamW(mini.parameters(), lr=config.lr)
train(config, mini, optimizer, train_loader, valid_loader)

  0%|          | 0/156727 [00:00<?, ?it/s]

  0%|          | 0/3630 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
import pandas as pd
from datasets import Dataset, DatasetDict

train_copd = pd.read_csv('COPDDS.csv')
valid_copd = pd.read_csv('COPDValidDS.csv')
test_copd = pd.read_csv('COPDTestDS.csv')
train_copd = train_copd.rename(columns={'uid1abstract': 'a', 'uid2abstract': 'b'})
valid_copd = valid_copd.rename(columns={'uid1abstract': 'a', 'uid2abstract': 'b'})
test_copd = test_copd.rename(columns={'uid1abstract': 'a', 'uid2abstract': 'b'})
train_copd['label'] = 1
test_copd['label'] = 1
copd_dataset = DatasetDict({
    'train': Dataset.from_pandas(train_copd),
    'valid': Dataset.from_pandas(valid_copd),
    'test': Dataset.from_pandas(test_copd)
})
train_cvd = pd.read_csv('CVDDS.csv')
valid_cvd = pd.read_csv('CVDValidDS.csv')
test_cvd = pd.read_csv('CVDTestDS.csv')
train_cvd = train_cvd.rename(columns={'uid1abstract': 'a', 'uid2abstract': 'b'})
valid_cvd = valid_cvd.rename(columns={'uid1abstract': 'a', 'uid2abstract': 'b', 'Label': 'label'})
test_cvd = test_cvd.rename(columns={'uid1abstract': 'a', 'uid2abstract': 'b', 'Label': 'label'})
train_cvd['label'] = 1
test_cvd['label'] = 1
cvd_dataset = DatasetDict({
    'train': Dataset.from_pandas(train_cvd),
    'valid': Dataset.from_pandas(valid_cvd),
    'test': Dataset.from_pandas(test_cvd)
})